In [1]:
import requests
OECD_ROOT_URL = 'http://stats.oecd.org/sdmx-json/data'

In [2]:
def make_OECD_request(dsname, dimensions, params=None, \
root_dir=OECD_ROOT_URL):
    """ Make a URL for the OECD API and return a response """
    if not params:
        params = {}
    dim_args = ['+'.join(d) for d in dimensions]
    dim_str = '.'.join(dim_args)
    url = root_dir + '/' + dsname + '/' + dim_str + '/all'
    print('Requesting URL: ' + url)
    return requests.get(url, params=params)

In [3]:
response = make_OECD_request('QNA',
(('USA', 'AUS'),('GDP', 'B1_GE'),('CUR', 'VOBARSA'), ('Q')),
{'startTime':'2009-Q1', 'endTime':'2010-Q1'})

Requesting URL: http://stats.oecd.org/sdmx-json/data/QNA/USA+AUS.GDP+B1_GE.CUR+VOBARSA.Q/all


In [4]:
response.status_code

200

In [5]:
data = response.json()
data.keys()

[u'header', u'dataSets', u'structure']

In [7]:
data

{u'dataSets': [{u'action': u'Information',
   u'series': {u'0:0:0:0': {u'attributes': [0, 0, 0, 0],
     u'observations': {u'0': [1318704.995857, None],
      u'1': [1326648.266393, None],
      u'2': [1330051.435134, None],
      u'3': [1339143.08436, None],
      u'4': [1345735.769993, None]}},
    u'1:0:0:0': {u'attributes': [0, 1, 0, 0],
     u'observations': {u'0': [14550598.411579, None],
      u'1': [14530900.721803, None],
      u'2': [14578392.803335, None],
      u'3': [14719519.766302, None],
      u'4': [14783232.581577, None]}}}}],
 u'header': {u'id': u'012401af-4fd0-4429-b17f-aad899c64db3',
  u'links': [{u'href': u'https://stats.oecd.org:443/sdmx-json/data/QNA/USA+AUS.GDP+B1_GE.CUR+VOBARSA.Q/all?endTime=2010-Q1&startTime=2009-Q1',
    u'rel': u'request'}],
  u'prepared': u'2018-07-18T02:32:57.8092665Z',
  u'sender': {u'id': u'OECD',
   u'name': u'Organisation for Economic Co-operation and Development'},
  u'test': False},
 u'structure': {u'annotations': [{u'text': u'',
  

In [8]:
REST_EU_ROOT_URL = "http://restcountries.eu/rest/v1"

In [9]:
def REST_country_request(field='all', name=None, params=None):
    headers={'User-Agent': 'Mozilla/5.0'}
    if not params:
        params = {}
    if field == 'all':
        return requests.get(REST_EU_ROOT_URL + '/all')
    url = '%s/%s/%s'%(REST_EU_ROOT_URL, field, name)
    print('Requesting URL: ' + url)
    response = requests.get(url, params=params, headers=headers)
    if not response.status_code == 200:
        raise Exception('Request failed with status code ' \
        + str(response.status_code))
    return response

In [10]:
response = REST_country_request('currency', 'usd')
response.json()

Requesting URL: http://restcountries.eu/rest/v1/currency/usd


[{u'alpha2Code': u'AS',
  u'alpha3Code': u'ASM',
  u'altSpellings': [u'AS',
   u'Amerika S\u0101moa',
   u'Amelika S\u0101moa',
   u'S\u0101moa Amelika'],
  u'area': 199.0,
  u'borders': [],
  u'callingCodes': [u'1684'],
  u'capital': u'Pago Pago',
  u'currencies': [u'USD'],
  u'demonym': u'American Samoan',
  u'gini': None,
  u'languages': [u'en', u'sm'],
  u'latlng': [-14.33333333, -170.0],
  u'name': u'American Samoa',
  u'nativeName': u'American Samoa',
  u'numericCode': u'016',
  u'population': 55519,
  u'region': u'Oceania',
  u'relevance': u'0.5',
  u'subregion': u'Polynesia',
  u'timezones': [u'UTC-11:00'],
  u'topLevelDomain': [u'.as'],
  u'translations': {u'de': u'Amerikanisch-Samoa',
   u'es': u'Samoa Americana',
   u'fr': u'Samoa am\xe9ricaines',
   u'it': u'Samoa Americane',
   u'ja': u'\u30a2\u30e1\u30ea\u30ab\u9818\u30b5\u30e2\u30a2'}},
 {u'alpha2Code': u'BQ',
  u'alpha3Code': u'BES',
  u'altSpellings': [u'BQ', u'Boneiru'],
  u'area': 294.0,
  u'borders': [],
  u'calling

In [12]:
from pymongo import MongoClient
def get_mongo_database(db_name, host='localhost',\
                       port=27017, username=None, password=None):
    """ Get named database from MongoDB with/out authentication """
    # make Mongo connection with/out authentication
    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s/%s'%\
        (username, password, host, db_name)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)
    return conn[db_name]

In [14]:
db_nobel = get_mongo_database('nobel_prize')
col = db_nobel['country_data']
response = REST_country_request()
col.insert_many(response.json())

In [15]:
res = col.find({'currencies':{'$in':['USD']}})
list(res)

[{u'_id': ObjectId('5b4ea854326b4f73fcf848e2'),
  u'alpha2Code': u'AS',
  u'alpha3Code': u'ASM',
  u'altSpellings': [u'AS',
   u'Amerika S\u0101moa',
   u'Amelika S\u0101moa',
   u'S\u0101moa Amelika'],
  u'area': 199.0,
  u'borders': [],
  u'callingCodes': [u'1684'],
  u'capital': u'Pago Pago',
  u'currencies': [u'USD'],
  u'demonym': u'American Samoan',
  u'gini': None,
  u'languages': [u'en', u'sm'],
  u'latlng': [-14.33333333, -170.0],
  u'name': u'American Samoa',
  u'nativeName': u'American Samoa',
  u'numericCode': u'016',
  u'population': 55519,
  u'region': u'Oceania',
  u'relevance': u'0.5',
  u'subregion': u'Polynesia',
  u'timezones': [u'UTC-11:00'],
  u'topLevelDomain': [u'.as'],
  u'translations': {u'de': u'Amerikanisch-Samoa',
   u'es': u'Samoa Americana',
   u'fr': u'Samoa am\xe9ricaines',
   u'it': u'Samoa Americane',
   u'ja': u'\u30a2\u30e1\u30ea\u30ab\u9818\u30b5\u30e2\u30a2'}},
 {u'_id': ObjectId('5b4ea854326b4f73fcf848f9'),
  u'alpha2Code': u'BQ',
  u'alpha3Code':